# Testing Bayesian Optimisation for Function Optimisation

### TODO
- rather than evaluating the objective at the same places, choose xs randomly

In [ ]:
# to automatically reload modules who's content has changed
%load_ext autoreload
%autoreload 2

# configure matplotlib
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

In [ ]:
import time
import numpy as np
import GPy
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

import ipywidgets as widgets
from IPython.display import display
from IPython.core.debugger import set_trace

In [ ]:
import function_bo as fbo
from testing import *

import sys
sys.path.append('..')

In [ ]:
plot_to_fit()

**by specifying a different kernel or providing a mean function, many different types of functions can be sampled**

In [ ]:
def plot_random_sample():
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(28, 6))
    
    bounds = ('x', xmin, xmax)
    op = fbo.Optimiser(None, bounds, 1)
    
    np.random.seed(0)
    f = op.select_random()
    plot_f(f, ax=ax1)
    ax1.set_title('default (RBF)')
    
    np.random.seed(0)
    mu = lambda x: 0.2*x + 0.3*np.sin(x)
    f = op.select_random(mu=mu)
    plot_f(f, ax=ax2)
    xs = np.linspace(xmin, xmax, num=100)
    ax2.plot(xs, mu(xs), label=r'\mu(x)')
    ax2.legend()
    ax2.set_title('default with mean function')
    
    f = op.select_random(GPy.kern.Matern52(input_dim=1, variance=1.0, lengthscale=0.1))
    plot_f(f, ax=ax3)
    ax3.set_title('Matern with small length scale')
plot_random_sample()

In [ ]:
def objective(f):
    # global reward
    R_g = integrate(lambda x: (f(x) - to_fit(x))**2, (xmin, xmax))
    # local rewards
    R_ls = []
    for x in np.linspace(xmin, xmax, num=10):
        R_l = (f(x)-to_fit(x))**2
        R_ls.append((x, R_l))
    return R_ls, R_g

bounds = ('x', xmin, xmax)
op = fbo.Optimiser(objective, bounds, desired_extremum='min', pre_phase_trials=10)

In [ ]:
np.random.seed(0)
op.run(11)

In [ ]:
plot_trials(op.trials)

In [ ]:
inc_i, inc = op.get_incumbent()
print('incumbent = trial {}'.format(inc_i))
plot_f(inc.f)

In [ ]:
plot_surrogate(op, op.trials[-1].surrogate)

In [ ]:
plot_surrogate_3D(op, op.trials[-1].surrogate)